In [1]:
import requests
import json
from IPython.display import JSON
from keys import *

In [ ]:
#be careful not to run too often as there is a maximum of 200 api calls per month

# URL para obtener vuelos del aeropuerto MMMX (CDMX)
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/MMMX/2025-09-01T08:00/2025-09-01T20:00"

# Pararaemtros de consulta de vuelos 
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

# Informacion que la API usa para identificar y autorizar 
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

 # Envia la solicitud a la API para obtener los datos usando la URL
response = requests.request("GET", url, headers=headers, params=querystring)

In [ ]:
# crear archivo json
if response.status_code == 200:
    new_data = response.json()

    try:
        #abre el archivo data.json para leer los datos que ya tiene
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    # Agrega los nuevos datos al final de los datos existentes ##
    existing_data.append(new_data)

        # Guarda la informacion actualizada en un archivo json 
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:

    # si se deetcta un error muetra el siguiente menesaje 
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [ ]:
flight_arrivals = response.json()

# Se itera sobre cada vuelo en la lista de llegadas
for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [ ]:
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

# Creamos un conjunto de los datos importantes del vuelo
def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })

# Convertimos los datos a DataFrame 
    return(flights_df)

# Sacamos datos de cada vuelo y los ponemos juntos en una tabla
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,UA 1024,Houston,United,Boeing 737-700,2025-09-01 14:35Z,2025-09-01 08:35-06:00
0,AM 505,Cancún,Aeromexico,Boeing 737-800,2025-09-01 14:30Z,2025-09-01 08:30-06:00
0,AM 905,Monterrey,Aeromexico,Boeing 737-900,2025-09-01 14:30Z,2025-09-01 08:30-06:00
0,AM 205,Guadalajara,Aeromexico,Boeing 737,2025-09-01 14:35Z,2025-09-01 08:35-06:00
0,VB 1355,Monterrey,VivaAerobus,Airbus A321 NEO,2025-09-01 14:40Z,2025-09-01 08:40-06:00
